경사하강법: 미분 가능한 함수의 최소값을 근사하기 위한 알고리즘의 일종. x값을 조금씩 변경해가며 함수값을 최소로 만드는 값 찾아가는데, 이 때 x값을 증가시킬지 감소시킬지 미분값을 통해 파악

학습률이 너무 높을 경우 global minimum이 아닌 local minimum에 빠질 수 있어요

선형회귀 
>

1.   데이터셋 생성 or 로드
2.   모델 생성, 초기화
3.   학습 루프 (예측값 도출, 손실함수값 계산, 미분값 계산, 경사하강법 업데이트 수행, 정해진 반복 횟수에 도달하거나 학습 종료 조건 만족 시 break)



## 다중 선형회귀
다중 선형회귀는 벡터 형태의 w 및 스칼라 형태의 b로 변수들을 생성한 후 행렬 형태의 학습 데이터에 대해서 행렬 연산으로 간편하게 예측값을 계산해낼 수 있음 

b는 브로드캐스팅 기능 활용, 이렇게 안하면 각각 변수 생성해야되고 독립 변수 숫자 늘면 번거로워짐

In [2]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
#import torch.optim as optim

In [12]:
#데이터셋 생성
x_train=t.FloatTensor([[73,80,75],[93,88,93],[89,91,80],[96,98,100]])
y_train=t.FloatTensor([[152],[185],[180],[196]])

#모델 초기화
W=t.zeros((3,1), requires_grad=True)
b=t.zeros(1, requires_grad=True)

optimizer= t.optim.SGD([W,b], lr=1e-5)
nb_epochs=1000

In [13]:
for epoch in range(nb_epochs+1):
  hypo=x_train.matmul(W) +b
  cost=t.mean((hypo-y_train)**2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  w_list=W.view(-1).tolist()
  if  epoch%100==0:
    print('Epoch {:4d}/{} w1: {:.3f}, w2: {:.3f}, w3: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, w_list[0], w_list[1], w_list[2], b.item(), cost.item()))

Epoch    0/1000 w1: 0.316, w2: 0.320, w3: 0.313, b: 0.004 Cost: 32036.250000
Epoch  100/1000 w1: 0.684, w2: 0.688, w3: 0.652, b: 0.008 Cost: 6.938233
Epoch  200/1000 w1: 0.694, w2: 0.693, w3: 0.637, b: 0.008 Cost: 6.573693
Epoch  300/1000 w1: 0.704, w2: 0.698, w3: 0.622, b: 0.008 Cost: 6.232265
Epoch  400/1000 w1: 0.714, w2: 0.702, w3: 0.608, b: 0.008 Cost: 5.912263
Epoch  500/1000 w1: 0.724, w2: 0.706, w3: 0.594, b: 0.008 Cost: 5.612178
Epoch  600/1000 w1: 0.733, w2: 0.710, w3: 0.581, b: 0.008 Cost: 5.330629
Epoch  700/1000 w1: 0.743, w2: 0.713, w3: 0.568, b: 0.008 Cost: 5.066350
Epoch  800/1000 w1: 0.752, w2: 0.717, w3: 0.555, b: 0.008 Cost: 4.818101
Epoch  900/1000 w1: 0.761, w2: 0.720, w3: 0.543, b: 0.008 Cost: 4.584816
Epoch 1000/1000 w1: 0.769, w2: 0.722, w3: 0.532, b: 0.008 Cost: 4.365375


In [14]:
test_data=t.FloatTensor([[73, 66, 70]])
prediction=test_data.matmul(W)+b
print(prediction)

tensor([[141.0700]], grad_fn=<AddBackward0>)


## torch.nn.Module을 이용해 클래스 형태로 다중 선형 회귀

In [20]:
import torch.nn as nn

class MultipleLinearRegressionModel(nn.Module): #nn.Module의 생성자 함수 호출
  def __init__(self):
    super().__init__() #super은 base class의 메서드를 활용한다는 의미
    self.linear=nn.Linear(3,1) #nn.linear(입력크기(독립변수3), 출력크기(1개 예측)), fully connected neural network 구성
  def forward(self, x): #입력 들어왔을 때 이렇게 계산해서 반환해라 ~, 오버라이딩이라 매개변수 개수 설정가능
    return self.linear(x)

In [23]:
import torch.nn.functional as F

x_train=t.FloatTensor([[73,80,75],[93,88,93],[89,91,80],[96,98,100]])
y_train=t.FloatTensor([[152],[185],[180],[196]])

model=MultipleLinearRegressionModel()
print(list(model.parameters())) #내부의 모든 가중치, 편향을 모아서 반환함

optimizer=t.optim.SGD(model.parameters(), lr=1e-5) #optimizer 선언 시 최적화 진행할 변수 입력
#초기화된 상태로 출력해도 zero tensor가 아님 왜지?
nb_epochs=1000
for epoch in range(nb_epochs+1):
  prediction=model(x_train) #model.forward(x_train)과 동일
  cost=F.mse_loss(prediction, y_train) #(예측값, 레이블), 파이토치에서 제공하는 MSE함수

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100==0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


[Parameter containing:
tensor([[ 0.2986, -0.3209,  0.1662]], requires_grad=True), Parameter containing:
tensor([-0.4653], requires_grad=True)]
Epoch    0/1000 Cost: 27988.988281
Epoch  100/1000 Cost: 10.650770
Epoch  200/1000 Cost: 9.864276
Epoch  300/1000 Cost: 9.137713
Epoch  400/1000 Cost: 8.466498
Epoch  500/1000 Cost: 7.846311
Epoch  600/1000 Cost: 7.273229
Epoch  700/1000 Cost: 6.743629
Epoch  800/1000 Cost: 6.254166
Epoch  900/1000 Cost: 5.801761
Epoch 1000/1000 Cost: 5.383488


In [24]:
test_data=t.FloatTensor([[73,66,70]])
prediction=model(test_data)
print(prediction)

tensor([[143.0529]], grad_fn=<AddmmBackward0>)


## logistic regression
이진분류, 로지스틱 회귀: 두개의 클래스 분류, 세개이상은 multi-class 분류라고함. 이름은 회귀지만 분류 문제에 더 많이 활용됨

가설의 형태는 ; h(x)=f(Wx+b)

1. sigmoid : 결과값은 [0,1], s자 곡선 형태로 단조 증가함. w값이 커질수록 그래프의 곡률이 커짐. b가 양수면 그래프가 왼쪽으로 음수면 오른쪽으로. 임계값(주로0.5)이상이냐 미만이냐고 0, 1로 클래스 구분
2. 로컬 미니멈이 다수라서MES말고 CE loss(cross entropy)라는 손실 함수를 씁니다요. 특정 클래스일 확률을 0~1로 나타냈을 때 실제 클래스와 얼마나 차이나는지를 정량화한 손실 함수의 일종 !

In [6]:
class LogisticRegression(nn.Module): #nn.Module이 모든 신경망모델의 base class인데
  def __init__(self):
    super().__init__() #그걸 상속받는거임(nn.Module의 매서드)
    self.linear=nn.Linear(2,1) ##############이부분은 두개 들어왔을 때 값 하나 예측
    self.sigmoid=nn.Sigmoid()

  def forward(self, x): #값 들어오면 어떻게 계산할것이냐.
    return self.sigmoid(self.linear(x)) #결과값 반환

In [14]:
x_train=t.FloatTensor([[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]])
y_train=t.FloatTensor([[0],[0],[0],[1],[1],[1]])

model=LogisticRegression()

optimizer=t.optim.SGD(model.parameters(), lr=1e-1)
nb_epochs=1000
for epoch in range(nb_epochs+1):
  prediction=model(x_train)
  cost=F.binary_cross_entropy(prediction, y_train) #이진분류 CE loss함수

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100==0:
    predicted_class=prediction>=t.FloatTensor([0,5]) #예측값이 0.5를 넘으면 class 1(True로 나타냄)로 간주
    correct_prediction=predicted_class.float()==y_train #예측값과 실제값이 일치하는 경우만 True 아니면 False
    accuracy=correct_prediction.sum().item()/len(correct_prediction) #정확도 계산
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy: {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy*100))

Epoch    0/1000 Cost: 1.142417 Accuracy: 100.00%
Epoch  100/1000 Cost: 0.395974 Accuracy: 100.00%
Epoch  200/1000 Cost: 0.333258 Accuracy: 100.00%
Epoch  300/1000 Cost: 0.288498 Accuracy: 100.00%
Epoch  400/1000 Cost: 0.253403 Accuracy: 100.00%
Epoch  500/1000 Cost: 0.225392 Accuracy: 100.00%
Epoch  600/1000 Cost: 0.202693 Accuracy: 100.00%
Epoch  700/1000 Cost: 0.184022 Accuracy: 100.00%
Epoch  800/1000 Cost: 0.168447 Accuracy: 100.00%
Epoch  900/1000 Cost: 0.155285 Accuracy: 100.00%
Epoch 1000/1000 Cost: 0.144032 Accuracy: 100.00%


In [15]:
test_data=t.FloatTensor([[5,5],[1,4]]) #class 1, class 0 하나씩 추론 시도
prediction=model(test_data)
print(prediction)


tensor([[0.9703],
        [0.0548]], grad_fn=<SigmoidBackward0>)
